In [ ]:
# ================================
# IMPORT LIBRARIES
# ================================
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, Average
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# ================================
# LOAD PRE-TRAINED MODELS
# ================================
dense_path = '/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/DenseNet-14-0.9663.hdf5'
mobilenet_path = '/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/mobileNetV2_without_seg_data_model.h5'
vgg16_path = '/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/VGG16-26-0.9423.hdf5'

model_1 = load_model(dense_path, compile=False)
model_2 = load_model(mobilenet_path, compile=False)
model_3 = load_model(vgg16_path, compile=False)

models = [model_1, model_2, model_3]

# ================================
# BUILD ENSEMBLE MODEL
# ================================
input_shape = (224, 224, 3)
model_input = Input(shape=input_shape)
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble_model_nonseg')

ensemble_model.compile(optimizer=Adam(learning_rate=0.001),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])


# ================================
# TRAIN ENSEMBLE MODEL
# ================================
history = ensemble_model.fit(train_generator,
                             validation_data=val_generator,
                             epochs=30)

# ================================
# SAVE MODEL
# ================================
model_save_dir = '/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/without_ensemble'
model_h5_path = os.path.join(model_save_dir, 'w_s_ensemble.hdf5')
model_tf_path = os.path.join(model_save_dir, 'ensemble_model5')

os.makedirs(model_save_dir, exist_ok=True)

# Save in HDF5 format
if os.path.exists(model_h5_path):
    os.remove(model_h5_path)
ensemble_model.save(model_h5_path)

# Save in TensorFlow SavedModel format
ensemble_model.save(model_tf_path)

print(f"Model saved successfully at:\n- {model_h5_path}\n- {model_tf_path}")

# ================================
# EVALUATE MODEL
# ================================
test_loss, test_accuracy = ensemble_model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# ================================
# PLOT ACCURACY & LOSS
# ================================
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', color='green')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', color='orange')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.ylim(0.8, 1.0)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', color='red')
plt.plot(history.history['val_loss'], label='Val Loss', color='blue')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, 0.5)
plt.legend()

plt.tight_layout()
plt.show()